# Banking Dataset - Análise exploratória

Os depósitos a prazo representam uma importante fonte de receita para os bancos. Trata-se de um investimento em dinheiro retido em uma instituição financeira, onde o montante é investido por um período fixo, a uma taxa de juros previamente acordada. Para maximizar a captação desses depósitos, os bancos implementam diversas estratégias de marketing direto, incluindo marketing por e-mail, publicidade, telemarketing e marketing digital.

Dentre essas estratégias, o telemarketing continua sendo um dos métodos mais eficazes para alcançar os clientes. No entanto, essa abordagem exige um grande investimento, pois grandes centrais de atendimento são contratadas para executar essas campanhas. Portanto, é crucial identificar previamente os clientes com maior probabilidade de conversão, de modo que possam ser especificamente alvo dessas campanhas.

Este notebook focará na análise exploratória dos dados relacionados às campanhas de marketing direto (chamadas telefônicas) de uma instituição bancária portuguesa.

Nas seções seguintes, exploraremos os dados disponíveis, realizaremos a limpeza e o pré-processamento necessários e, finalmente, analisaremos diversas características dos clientes para identificar padrões que impactam a chance de um cliente efetuar o depósito, mensurado pela nossa variável resposta _y_. 

A fonte dos dados é [_Banking Dataset - Marketing Targets_](https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets) do Kaggle.

# Índice

- [1.0 Dicionário de dados](#1_0)
- [1.1 Configurações e bibliotecas](#1_1)

# <a id='1_0'></a> 1.1 Dicionário de dados

| Variável  | Tipo de Variável   | Descrição                                                |
|-----------|--------------------|----------------------------------------------------------|
| age       | Numérica Discreta  | Idade do cliente.                                        |
| job       | Categórica Nominal | Tipo de trabalho do cliente.                             |
| marital   | Categórica Nominal | Estado civil do cliente.                                 |
| education | Categórica Ordinal | Nível de educação do cliente.                            |
| default   | Categórica Nominal | Cliente tem crédito em default.                          |
| balance   | Numérica Contínua  | Saldo anual médio, em euros.                             |
| housing   | Categórica Nominal | Cliente tem empréstimo imobiliário.                      |
| loan      | Categórica Nominal | Cliente tem empréstimo pessoal.                          |
| contact   | Categórica Nominal | Tipo de comunicação de contato.                          |
| day       | Categórica Ordinal | Dia do último contato.                                   |
| month     | Categórica Ordinal | Mês do último contato.                                   |
| duration  | Numérica Contínua  | Duração do último contato, em segundos.                  |
| campaign  | Numérica Discreta  | Número de contatos realizados durante esta campanha.     |
| pdays     | Numérica Discreta  | Dias desde o último contato de uma campanha anterior.    |
| previous  | Numérica Discreta  | Número de contatos realizados antes desta campanha.      |
| poutcome  | Categórica Nominal | Resultado da campanha de marketing anterior.             |
| y         | Categórica Nominal | Cliente fez um depósito dentro do prazo.                 |


# <a id='1_1'></a> 1.2 Configurações e bibliotecas

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt